# Gurobi

Gurobi es un solver de optimización comercial, incluye:

- Linear programming (LP)
- Quadratic programming (QP)
- Quadratically constrained programming (QCP)
- Mixed integer linear programming (MILP)
- Mixed-integer quadratic programming (MIQP)
- Mixed-integer quadratically constrained programming (MIQCP).
- Non-Convex Quadratic Optimization (NCQO)

Hay varias formas de programar un modelo de optimización usando Gurobi en Julia, ejemplos en:
- https://github.com/JuliaOpt/Gurobi.jl

Benchmark:
- https://www.gurobi.com/pdfs/benchmarks.pdf

Se recomienda usar Gurobi con JuMP, <a href="http://www.juliaopt.org/JuMP.jl/stable/">JuMP</a> es un lenguage de optimización matemática que soporta un gran abanico de solvers.

In [1]:
#using Pkg
#Pkg.add(PackageSpec(url="https://github.com/JuliaOpt/JuMP.jl/", rev="master"))
#Pkg.add("Gurobi")
#Pkg.add("GLPK") #solver
#Pkg.add("Test") #librería para testear
#Pkg.add("DataFrames")
#Pkg.add("CSV")
using JuMP, Gurobi, GLPK, Test, DataFrames, CSV

# 1. Simple LP

\begin{equation}
\begin{matrix}
\underset{x,y}{\min} & x+y \\
\textrm{s.t.} & 50x+24y & \leq & 2400 \\
& 30x+33y & \leq & 2100\\
& x & \geq & 45\\
& y & \geq & 5\\
\end{matrix}
\end{equation}

Presolve: operaciones que realiza un solver para transformar un problema de optimización en uno equivalente más fácil de resolver.

In [2]:
#Creamos el objeto modelo
simple_model = Model()

#Le indicamos a JuMP que el solver a utilizar es Gurobi
set_optimizer(simple_model, Gurobi.Optimizer)
#set_optimizer(simple_model, GLPK.Optimizer) #GLPK solver

#Seteamos opciones del solver, en este caso desactivamos el Presolve
set_optimizer_attributes(simple_model, "Presolve" => 0) #comentar esta línea si se usa GLPK 

#declaración de variables de decisión
@variable(simple_model, x>=5,  base_name="x")
@variable(simple_model, y>=45, base_name="y")

#restricciones, el nombre de la restricción es opcional
@constraint(simple_model, const1, 50x + 24y<=2400)
@constraint(simple_model, const2, 30x + 33y<=2100)

#función objetivo
@objective(simple_model, Min, x+y)

#resolver
optimize!(simple_model)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x1e20dfff
Coefficient statistics:
  Matrix range     [2e+01, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 4e+01]
  RHS range        [2e+03, 2e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  5.000000000e+01


In [3]:
simple_model

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: const1, const2, x, y

In [4]:
#acceder a una variable del modelo
simple_model[:x]

x

In [5]:
x

x

In [6]:
#acceder a una restricción del modelo
simple_model[:const1]

const1 : 50 x + 24 y ≤ 2400.0

In [7]:
const1

const1 : 50 x + 24 y ≤ 2400.0

In [8]:
#preguntar por qué el solver paró
termination_status(simple_model)

OPTIMAL::TerminationStatusCode = 1

In [9]:
#verifical el estado de la solución primal
primal_status(simple_model)

FEASIBLE_POINT::ResultStatusCode = 1

In [10]:
#verifical el estado de la solución dual 
dual_status(simple_model)

FEASIBLE_POINT::ResultStatusCode = 1

In [11]:
#Preguntar el valor óptimo, la solución primal, solución dual y valores de las restricciones.
println("Optimal objective: ", objective_value(simple_model), "\n x = ",value(simple_model[:x]), 
    ", y = ", value(simple_model[:y]), "\n dual const1 = ", dual(simple_model[:const1]), 
    ", dual const2 = ", dual(simple_model[:const2]), "\n const1 = ", value(simple_model[:const1]), 
    ", const2 = ", value(simple_model[:const2]))

Optimal objective: 50.0
 x = 5.0, y = 45.0
 dual const1 = 0.0, dual const2 = 0.0
 const1 = 1330.0, const2 = 1635.0


In [12]:
#veremos que pasa si obligamos al que problema primal sea no acotado
delete_lower_bound(simple_model[:x])#eliminamos la cota inferior de la variable
println("x tiene cota inferior? ", has_lower_bound(x))
#resolver
optimize!(simple_model)

x tiene cota inferior? false
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [2e+01, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+01, 4e+01]
  RHS range        [2e+03, 2e+03]
       0      handle free variables                          0s

Solved in 1 iterations and 0.00 seconds
Unbounded model


In [13]:
termination_status(simple_model)

DUAL_INFEASIBLE::TerminationStatusCode = 3

In [14]:
primal_status(simple_model)

INFEASIBILITY_CERTIFICATE::ResultStatusCode = 4

In [15]:
dual_status(simple_model)

NO_SOLUTION::ResultStatusCode = 0

In [16]:
objective_value(simple_model) #-∞

-3.2e29

Como el primal es no acotado (valor óptimo = $-\infty$) el dual es infactible.

# 2. Diet Problem

Supongamos que hay $n$ diferentes comidas y $m$ diferentes nutrientes, y tenemos la siguiente tabla de contenido nutricial por cada unidad de comida:

|            |  food 1  | ... |  food n  |
|:----------:|:--------:|:---:|:--------:|
| nutrient 1 | $a_{11}$ | ... | $a_{1n}$ |
|   &#8942;  |  &#8942; |     |  &#8942; |
| nutrient m | $a_{m1}$ | ... | $a_{mn}$ |

Sea $A$ una matrix $mxn$ con entradas $a_{ij}$, notar que la columna $A_{j}$ representa el contenido nutricional de la comida $j$, sea $b$ un vector con los requirimientos nutricionales de una dieta ideal, sea $c$ el vector de costos de cada unidad de comida. El problema anterior se puede representar como LP en forma estándar:

\begin{equation}
\begin{matrix}
\underset{x}{\min} & c^{´}x \\
\textrm{s.t.} & Ax = b\\
& x \geq 0\\
\end{matrix}
\end{equation}

Es decir se busca la dieta que satisfaga los requirimientos nutricionales de una dieta ideal a costo mínimo.

In [17]:
#Creamos una instancia

#nutrientes
nutrients = ["calorías", "proteína", "grasa", "carbohidratos"]

#comidas
foods = ["hamburguesa", "completo italiano", "papas fritas", "pizza", "fideos con salsa",
         "porotos con mazamorra", "lentejas", "pastel de choclo", "platano", "leche",
         "yogurt", "batido"]

#requerimientos dieta ideal
b = JuMP.Containers.DenseAxisArray(
    [
    2000, #kcal
    120, #proteínas [gr]
    40, #grasas [gr]
    250 #carbohidratos [gr] 
    ], nutrients)

#vector de costos en pesos chilenos por unidad de alimento
c = JuMP.Containers.DenseAxisArray(
    [
    1400, #hamburguesa
    1450, #completo italiano
    550, #papas fritas
    750, #pizza
    2500, #fideos con salsa
    2000, #porotos con mazamorra
    1500, #lentejas
    3000, #pastel de choclo
    300, #platano
    250, #leche
    400, #yogurt
    600 #batido
    ], foods)

#matriz A transpuesta, (comida, nutriente [gr])
At = JuMP.Containers.DenseAxisArray(
    [
    482 21 26 41; #hamburguesa 
    440 12 24 44; #completo italiano 1450
    502 4 22 72; #papas fritas 550
    612.5 32.5 40.1 30.4; #pizza 750
    329 20 13 33; #fideos con salsa 
    336 14 12 43; #porotos con mazamorra
    358.8 27 1.2 60; #lentejas
    567 21 27 60; #pastel de choclo
    204.2 1.9 0.6 47.8; #platano
    71.6 9 0.4 8; #leche
    92.4 14 0.8 7.3; #yogurt
    230 23 0 8; #batido
    ], foods, nutrients);

In [18]:
b["calorías"]

2000

In [19]:
c["hamburguesa"]

1400

In [20]:
At["hamburguesa", "calorías"]

482.0

In [21]:
#guardamos la instancia en un diccionario
diet_data = Dict("b"=>b, "c"=>c, "At"=>At)

Dict{String,JuMP.Containers.DenseAxisArray} with 3 entries:
  "c"  => 1-dimensional DenseAxisArray{Int64,1,...} with index sets:…
  "b"  => 1-dimensional DenseAxisArray{Int64,1,...} with index sets:…
  "At" => 2-dimensional DenseAxisArray{Float64,2,...} with index sets:…

In [22]:
function diet_model(solver, diet_data, verbose)
    """
    Diet model
    Input
    - solver: MathOptInterface.AbstractOptimizer, 
        recibe un solver de optimización, por ejemplo, Gurobi.Optimizer.
    - diet_data: Dict{String, JuMP.Containers.DenseAxisArray}
        recibe un diccionario de la forma {"c"=>c, "b"=>b, "At"=>At},
        donde b, c y At son objetos JuMP.Containers.DenseAxisArray,
        con dimensiones 1, 1 y 2 respectivamente. Además debe cumplirse:
        - size(At) = (n,m), donde m = length(b), n=length(c). 
        - c>0, b>0, At>=0
    - verbose = Bool
        controla la verbosidad del programa.
    Output: AbstractModel
        retorna el objeto modelo que almacena el valor óptimo y las soluciones óptimas en caso de 
        ser alcanzadas.
    """
   
    #obtener parámetros de diet_data y guardar en variables
    b, c, At = diet_data["b"], diet_data["c"], diet_data["At"]
    
    #testear ingreso correcto de parámetros
    @test size(At) == (length(c), length(b)) #testear dimensiones
    #testear positividad de b,c y no negatividad de At
    @test sum(b.data.> 0)==length(b) && sum(c.data.> 0)==length(c) && sum(At.data.>= 0)==length(At)
    
    #lista de comidas y nutrientes
    foods, nutrients = At.axes
    
    #crear modelo y setear solver
    model = Model(solver)
    
    #cantidad a consumir por comida
    @variable(model, x[foods]>=0)
    
    #restricciones por nutriente, la suma de los nutrientes por alimento debe ser igual al requerimiento ideal
    @constraint(model, [i in nutrients],
        sum(At[j,i]*x[j] for j in foods)==b[i]
    )
    
    #Minimizar costo de la dieta ideal
    @objective(model, Min, sum(c[j]*x[j] for j in foods))
    #Resolver
    optimize!(model)
    
    #Status de término del solver
    term_status = termination_status(model)
    is_optimal = term_status == MOI.OPTIMAL
    
    if verbose
        #Si se alcanzó el óptimo mostrar solución óptima, sino mostrar mensaje que no se alcanzó
        if is_optimal
            for food in foods
                println("$(food) = $(value(x[food]))")
            end
        else
            println("El solver no encontró una solución óptima")
        end
    end
    return model
end

diet_model (generic function with 1 method)

In [23]:
idiet_model = diet_model(GLPK.Optimizer, diet_data, true);

hamburguesa = 0.0
completo italiano = 0.0
papas fritas = 0.0
pizza = 0.8902237722884057
fideos con salsa = 0.0
porotos con mazamorra = 0.0
lentejas = 0.0
pastel de choclo = 0.0
platano = 3.486625446916778
leche = 5.525128657712165
yogurt = 0.0
batido = 1.5094339622641502


In [24]:
idiet_model = diet_model(Gurobi.Optimizer, diet_data, true);

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 4 rows, 12 columns and 47 nonzeros
Model fingerprint: 0xff66419a
Coefficient statistics:
  Matrix range     [4e-01, 6e+02]
  Objective range  [2e+02, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 2e+03]
Presolve time: 0.01s
Presolved: 4 rows, 12 columns, 47 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7692308e+03   1.506731e+02   0.000000e+00      0s
       4    4.0005980e+03   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds
Optimal objective  4.000598005e+03
hamburguesa = 0.0
completo italiano = 0.0
papas fritas = 0.0
pizza = 0.8902237722884059
fideos con salsa = 0.0
porotos con mazamorra = 0.0
lentejas = 0.0
pastel de choclo = 0.0
platano = 3.4866254469167774
leche = 5.525128657712159
yogurt = 0.0
batido = 1.5094339622641

# 3. Piecewise linear function and Data fitting

## Piecewise linear function

Una función lineal por partes se puede representar como:

$f(x) = \underset{i=1,\ldots,m}{max}(c_{i}^{´}+d_{i})$

Donde $x, c_{i} \in \rm I\!R^{n}, d_{i} \in \rm I\!R,  \forall  i=1,\ldots,m$. Notar que esta función es convexa, de hecho se puede demostrar que el máximo de funciones convexas es convexo. 

Supongamos que nos enfrentamos al problema de buscar el $x$ que minimize $f(x)$:

$\underset{x}{min} \underset{i=1,\ldots,m}{max}(c_{i}^{´}+d_{i})$

El problema anterior se puede escribir como un LP:

\begin{equation}
\begin{matrix}
\underset{x, z}{\min} & z \\
\textrm{s.t.} & z & \geq & c^{´}_{i}x+d_{i} & i=1,\ldots,m\\
\end{matrix}
\end{equation}

## Data fitting
Supongamos que contamos con $m$ observaciones de la forma ($x_{i}, y_{i}$), $i=1,\ldots,m$, donde $x_{i} \in \rm I\!R^{n}$ y $y_{i}\in\rm I\!R$, deseamos construir un modelo que prediga el valor de $y$ dado $x$. En tal caso usualmente se utiliza un modelo lineal de la forma $y=\beta^{´}x$, donde $\beta$ es un vector de parámetros a estimar. Sea $|y_{i}-\beta^{´}x_{i}|$ el residuo o error de predicción en la observación $i$, generalmente se busca determinar el vector de parámetros que minimiza el error cuadrático medio, pero en este caso buscaremos el vector de parámetros que minimice el residuo más grande:

$\underset{\beta}{min} \underset{i=1,\ldots,m}{max}|y_{i}-\beta^{'}x_{i}|$

Notar que la función modulo es una función lineal por partes, ya que $|x|=max\{x,-x\}$. El problema anterior se puede reescribir como un LP:

\begin{equation}
\begin{matrix}
\underset{\beta, z}{\min} & z \\
\textrm{s.t.} & y_{i}-\beta^{'}x_{i} & \leq z & i=1,\ldots,m\\
& -y_{i}+\beta^{'}x_{i} & \leq z & i=1,\ldots, m
\end{matrix}
\end{equation}

Para el ejemplo se utilizará la data `01.Housing.csv`, la base de datos posee precios de casas de algunas localidades de USA junto a un par de covariables que se correlacionan con este.

In [25]:
#ajustar variable de entorno para ver todas las columnas del dataframe
ENV["COLUMNS"]=150

150

In [26]:
df = CSV.read("01.Housing.csv");

In [27]:
#importar .csv y crear un dataframe que lo almacena
df = CSV.read("01.Housing.csv");
df.bias = [1 for i in 1:size(df)[1]]
#printear primeras 5 filas del dataframe
first(df, 5)

,Avg Area Income,Avg Area House Age,Avg Area Number of Rooms,Avg Area Number of Bedrooms,Area Population,Price,bias
,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,79545.5,5.68286,7.00919,4.09,23086.8,1.05903e6,1
2,79248.6,6.0029,6.73082,3.09,40173.1,1.50589e6,1
3,61287.1,5.86589,8.51273,5.13,36882.2,1.05899e6,1
4,63345.2,7.18824,5.58673,3.26,34310.2,1.26062e6,1
5,59982.2,5.04055,7.83939,4.23,26354.1,6.30943e5,1


In [28]:
columns = String.(names(df))
setdiff!(columns, ["Price"])

6-element Array{String,1}:
 "Avg Area Income"            
 "Avg Area House Age"         
 "Avg Area Number of Rooms"   
 "Avg Area Number of Bedrooms"
 "Area Population"            
 "bias"                       

In [29]:
features = [Symbol(col) for col in columns]
target = Symbol("Price")
#vector de características
x = convert(Matrix, df[:,features]);
#variable objetivo a predecir
y = df[:,target];
#almacenamos la data en un diccionario
data = Dict("x"=>x, "y"=>y)

Dict{String,Array{Float64,N} where N} with 2 entries:
  "x" => [79545.5 5.68286 … 23086.8 1.0; 79248.6 6.0029 … 40173.1 1.0; … ; 68001.3 5.53439 … 42625.6 1.0; 65510.6 5.99231 … 46501.3 1.0]
  "y" => [1.05903e6, 1.50589e6, 1.05899e6, 1.26062e6, 6.30943e5, 1.06814e6, 1.50206e6, 1.57394e6, 7.9887e5, 1.54515e6  …  4.79501e5, 1.26372e6, 1.568…

In [30]:
function fit_model(solver, data, verbose)
    """
    Data fitting model
    Input
    - solver: MathOptInterface.AbstractOptimizer, 
        recibe un solver de optimización, por ejemplo, Gurobi.Optimizer.
    - diet_data: Dict{String, Array}
        recibe un diccionario de la forma {"x"=>x, "y"=>y},
        donde size(x) = (m,n) y size(y) = m
    - verbose = Bool
        controla la verbosidad del programa.
    Output: AbstractModel
        retorna el objeto modelo que almacena el valor óptimo y las soluciones óptimas en caso de 
        ser alcanzadas.
    """
    #obtener datos
    x, y = data["x"], data["y"]
    m, n = size(x) 
    
    #crear modelo
    model = Model(solver)
    
    #variables
    @variable(model, β[1:n])
    @variable(model, z)
    
    #restricciones
    for i in 1:m
        @constraints(model, begin
            y[i]-sum(β[j]*x[i,j] for j in 1:n)<=z
            -y[i]+sum(β[j]*x[i,j] for j in 1:n)<=z
        end)
    end
    @objective(model, Min, z)
    
    optimize!(model)
    
    #Status de término del solver
    term_status = termination_status(model)
    is_optimal = term_status == MOI.OPTIMAL
    
    if verbose
        #Si se alcanzó el óptimo mostrar solución óptima, sino mostrar mensaje que no se alcanzó
        if is_optimal
            println("Valor óptimo: $(objective_value(model))")
            for j in 1:n
                println("β[$(j)] = $(value(β[j]))")
            end
        else
            println("El solver no encontró una solución óptima")
        end
    end
    return model
    return model
end

fit_model (generic function with 1 method)

In [31]:
ifit_model = fit_model(Gurobi.Optimizer, data, true);

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 10000 rows, 7 columns and 70000 nonzeros
Model fingerprint: 0x11c32863
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 2e+06]
Presolve time: 0.02s
Presolved: 7 rows, 10000 columns, 70000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.2989505e+06   1.028341e+03   2.803551e+10      0s
     152    3.2644041e+05   0.000000e+00   0.000000e+00      0s

Solved in 152 iterations and 0.03 seconds
Optimal objective  3.264404085e+05
Valor óptimo: 326440.4084616313
β[1] = 19.621117214360737
β[2] = 162871.91992379463
β[3] = 98456.2742777419
β[4] = 10361.863959013777
β[5] = 15.166913536089698
β[6] = -2.3730902005351493e6


In [32]:
ifit_model = fit_model(GLPK.Optimizer, data, true);

Valor óptimo: 326440.4084616317
β[1] = 19.621117214360716
β[2] = 162871.91992379466
β[3] = 98456.27427774173
β[4] = 10361.863959013728
β[5] = 15.166913536089693
β[6] = -2.373090200535146e6


In [33]:
β = [value(elem) for elem in ifit_model[:β]]

6-element Array{Float64,1}:
     19.621117214360716  
 162871.91992379466      
  98456.27427774173      
  10361.863959013728     
     15.166913536089693  
     -2.373090200535146e6

In [34]:
residuals  = [abs(y[i]-β'*x[i,:]) for i in 1:5000];

In [35]:
value(ifit_model[:z]) ≈ maximum(residuals)

true